Import packages and load rotation functions

In [74]:
# import the necessary packages
#Life is good
from PIL import Image
import pytesseract
import numpy as np
#from pytesseract import image_to_string
#from pytesseract import image_to_boxes
#from pytesseract import image_to_data
import argparse
import cv2
import os

def orientationScore(image,skipStep = 10):
    totValue = 0
    for row in range(len(image)):
        rowVal = 0
        for col in range(0,len(image[0]),skipStep):
            rowVal += image[row][col]
        totValue += rowVal*rowVal
    return totValue

def findOrient(image,angleRange = 5,skipStep = 10):
    maxValue = -1
    maxRot  = 0
    imageRows, imageCols = image.shape
    accuracy = 0.1


    
    maxLen = angleRange*2*int(1/accuracy)
    #print maxLen
    for i in range(-1, maxLen):
        #rot = i
        rot = (round(i / 2 - 0.4) + 1) * ((-1) * (i % 2) + ((i + 1) % 2)) * 0.1
        #print rot
        # print("Rotation: ",rot,". i: ",i)
        M = cv2.getRotationMatrix2D((imageCols / 2, imageRows / 2), rot, 1)
        dst = cv2.warpAffine(image, M, (imageCols, imageRows), borderValue=(255, 255, 255))

        value = orientationScore(255-dst,skipStep = 10)
        if value>maxValue:
            maxValue = value
            maxRot = rot
        if i%int(maxLen/100)==0:
            print "Percentage complete: ", round(i*100.0/maxLen),"%. Current angle (degree): ",rot
        #print "Current angle: ",rot,". Current value: ", value,". Max value: ",maxValue,". Max rot: ",maxRot
        #cv2.imshow("Rotated Image: ",dst)
        #cv2.waitKey(0)

    M = cv2.getRotationMatrix2D((imageCols / 2, imageRows / 2), maxRot, 1)
    dst = cv2.warpAffine(image, M, (imageCols, imageRows), borderValue=(255, 255, 255))
    #cv2.imshow("Rotated Image: ", dst)
    #cv2.waitKey(0)
    return dst

Specify the paramaters of the OCR run

In [75]:
args = {}
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/rotate.png"
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/rotatedDistorted.png"
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/RealScan.jpg"
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/phonePhoto.jpg"
args["image"] = "/home/dries/devel/Praelexis/OCR/Images/bankRotatedBackround.jpg"

args["preprocess"] = ""
args["rotationSearch"] = 5
con ="-psm 1 --oem 2 -c tessedit_char_whitelist=0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLNMOPQRSTUVWXYZ-+,.()&:_/\*"
lan = "afr"

Main code

In [76]:

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,help="path to input image to be OCR'd")
# ap.add_argument("-p", "--preprocess", type=str, default="thresh",	help="type of preprocessing to be done")
# args = vars(ap.parse_args())s


# load the example image and convert it to grayscale
image = cv2.imread(args["image"])

# cv2.imshow("Image", image)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite('/home/dries/devel/Praelexis/OCR/Output/Original.png',gray)
print("Finding orientation..")
gray = findOrient(gray,angleRange = args["rotationSearch"],skipStep = 10)
print("Image corrected. Scanning image..")
# check to see if we should apply thresholding to preprocess the
# image
if args["preprocess"] == "thresh":
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# make a check to see if median blurring should be done to remove
# noise
elif args["preprocess"] == "blur":
    gray = cv2.medianBlur(gray, 1)

'''cv2.namedWindow('Input', cv2.WINDOW_NORMAL)
cv2.imshow("Input", gray)    
cv2.waitKey(0)'''



# write the grayscale image to disk as a temporary file so we can
# apply OCR to it
#filename = "{}.png".format(os.getpid())
#cv2.imwrite(filename, gray)

# load the image as a PIL/Pillow image, apply OCR, and then delete
# the temporary file
#text = pytesseract.image_to_string(gray,boxes=False)


# This might not be the best approach. I think the system also directly searches for words.
# These word phrases are not included in the config settings.

cv2.imwrite('/home/dries/devel/Praelexis/OCR/Output/preProcessed.png',gray)

text = pytesseract.image_to_string(gray,lang=lan,boxes=False,config=con)

f = open('/home/dries/devel/Praelexis/OCR/Output/outputText.txt','w')
f.write(text)
f.close()

#print "Text: ", text

Finding orientation..
Percentage complete:  -1.0 %. Current angle (degree):  -0.0
Percentage complete:  0.0 %. Current angle (degree):  0.1
Percentage complete:  1.0 %. Current angle (degree):  -0.1
Percentage complete:  2.0 %. Current angle (degree):  0.2
Percentage complete:  3.0 %. Current angle (degree):  -0.2
Percentage complete:  4.0 %. Current angle (degree):  0.3
Percentage complete:  5.0 %. Current angle (degree):  -0.3
Percentage complete:  6.0 %. Current angle (degree):  0.4
Percentage complete:  7.0 %. Current angle (degree):  -0.4
Percentage complete:  8.0 %. Current angle (degree):  0.5
Percentage complete:  9.0 %. Current angle (degree):  -0.5
Percentage complete:  10.0 %. Current angle (degree):  0.6
Percentage complete:  11.0 %. Current angle (degree):  -0.6
Percentage complete:  12.0 %. Current angle (degree):  0.7
Percentage complete:  13.0 %. Current angle (degree):  -0.7
Percentage complete:  14.0 %. Current angle (degree):  0.8
Percentage complete:  15.0 %. Curren

Now lets generate a output image of what the system sees.

In [77]:
data = pytesseract.image_to_string(gray,boxes=True,config=con)
lines = data.split('\n')

array = []


yLen = len(gray)
newPage = gray.copy()

for i in range(len(newPage)):
    for j in range(len(newPage[0])):
        newPage[i][j] = 255

for line in lines:
    entry = line.split(' ')
    #print(entry)
    array.append(entry)
    cv2.rectangle(gray, (int(entry[1]), yLen-int(entry[2])), (int(entry[3]), yLen-int(entry[4])), (0,0,0), 1)
    font = cv2.FONT_HERSHEY_SIMPLEX
    #print("Here: ",entry[0], " ", len(entry[0]))
    try:
        char = str(entry[0])
        #print(char)
        
        xSize = 0.5*(int(entry[3])-int(entry[1]))
        ySize = 0.5*(int(entry[4])-int(entry[2]))
        
        size = xSize
        
        if ySize>xSize:
            size = ySize
        
        fondSize = size/15.0
        loc = (int(entry[1])+int(0.5*(int(entry[3])-int(entry[1]))),yLen-int(entry[2])+int(0.5*(int(entry[4])-int(entry[2]))))
        cv2.putText(newPage,char,loc,font, fondSize,(0,0,255))
        #print char," ",fondSize, " ", loc
    except (TypeError, ValueError):
        print "Not added: ",entry
        xSize = 0.5*(int(entry[3])-int(entry[1]))
        ySize = 0.5*(int(entry[4])-int(entry[2]))
        
        size = xSize
        
        if ySize>xSize:
            size = ySize
        
        fondSize = size/15.0
        loc = (int(entry[1])+int(0.5*(int(entry[3])-int(entry[1]))),yLen-int(entry[2])+int(0.5*(int(entry[4])-int(entry[2]))))
        cv2.putText(newPage,'???',loc,font, fondSize,(0,0,255))
        
        
#data = pytesseract.image_to_boxes(gray)
#os.remove(filename)
#print "Text found:\n",text

#print "Data:\n",array
# show the output images

#for entry in data:
    #print "Entry: ", entry
    #cv2.rectangle(gray, (100, 100), (1000, 1000), (0,0,0), 20)
#cv2.namedWindow('Input', cv2.WINDOW_NORMAL)
#cv2.imshow("Input", gray)
cv2.imwrite('/home/dries/devel/Praelexis/OCR/Output/Boxes.png',gray)
cv2.imwrite('/home/dries/devel/Praelexis/OCR/Output/Estimation.png',newPage)

~   69.2666666667   (1245, 245)
0   0.866666666667   (227, 315)
1   0.8   (244, 313)
-   0.333333333333   (259, 297)
0   0.833333333333   (274, 314)
3   0.866666666667   (291, 315)
-   0.333333333333   (306, 297)
2   0.8   (321, 313)
0   0.833333333333   (339, 314)
1   0.8   (356, 313)
6   0.866666666667   (374, 315)
B   0.833333333333   (597, 314)
R   0.833333333333   (622, 314)
N   0.833333333333   (646, 314)
-   0.333333333333   (665, 297)
B   0.833333333333   (683, 314)
Y   0.833333333333   (708, 314)
C   0.833333333333   (742, 314)
A   0.8   (767, 314)
S   0.833333333333   (790, 314)
H   0.833333333333   (813, 314)
B   0.833333333333   (847, 314)
Y   0.833333333333   (872, 314)
C   0.833333333333   (906, 314)
A   0.833333333333   (931, 314)
S   0.833333333333   (953, 314)
H   0.833333333333   (976, 314)
1   0.833333333333   (1691, 314)
0   0.833333333333   (1709, 314)
0   0.833333333333   (1728, 314)
0   0.833333333333   (1745, 314)
0   0.866666666667   (1763, 315)
.   0.166666666

u   0.566666666667   (1879, 1183)
n   0.566666666667   (1897, 1183)
t   0.733333333333   (1912, 1186)
,   0.3   (1920, 1184)
i   0.866666666667   (1939, 1188)
t   0.733333333333   (1949, 1186)
w   0.8   (1976, 1183)
i   0.866666666667   (1994, 1188)
l   0.866666666667   (2004, 1188)
l   0.866666666667   (2013, 1188)
b   0.866666666667   (2036, 1188)
e   0.566666666667   (2053, 1183)
t   0.733333333333   (2075, 1186)
w   1.06666666667   (2097, 1188)
e   0.566666666667   (2121, 1183)
n   0.6   (2138, 1184)
t   0.733333333333   (2161, 1186)
h   0.866666666667   (2175, 1188)
a   0.6   (2192, 1184)
t   0.7   (2205, 1184)
h   0.833333333333   (228, 1222)
e   0.6   (245, 1219)
/   0.833333333333   (257, 1222)
s   0.566666666667   (269, 1218)
h   0.833333333333   (285, 1222)
e   0.566666666667   (302, 1218)
h   0.833333333333   (327, 1222)
a   0.566666666667   (345, 1218)
s   0.6   (359, 1219)
f   0.833333333333   (383, 1222)
o   0.566666666667   (395, 1218)
u   0.566666666667   (413, 1218)
n 

n   0.566666666667   (2174, 1428)
g   0.833333333333   (2193, 1440)
f   0.833333333333   (229, 1467)
r   0.566666666667   (238, 1463)
o   0.6   (253, 1465)
m   0.866666666667   (276, 1463)
y   0.8   (307, 1475)
o   0.6   (324, 1465)
u   0.566666666667   (343, 1464)
s   0.6   (367, 1465)
h   0.866666666667   (383, 1469)
a   0.6   (400, 1465)
r   0.566666666667   (414, 1464)
i   0.866666666667   (424, 1469)
n   0.566666666667   (438, 1464)
y   1.1   (461, 1479)
d   0.866666666667   (483, 1469)
i   0.866666666667   (496, 1469)
s   0.6   (508, 1465)
c   0.566666666667   (523, 1464)
l   0.833333333333   (536, 1468)
o   0.6   (550, 1466)
s   0.6   (566, 1466)
i   0.866666666667   (578, 1469)
n   0.566666666667   (592, 1464)
g   0.833333333333   (610, 1476)
o   0.6   (636, 1466)
f   0.833333333333   (654, 1468)
y   0.833333333333   (675, 1476)
o   0.6   (693, 1466)
u   0.6   (711, 1466)
r   0.566666666667   (726, 1464)
l   0.833333333333   (746, 1468)
o   0.6   (760, 1466)
g   0.833333333333 

t   0.7   (917, 1775)
e   0.566666666667   (930, 1773)
m   0.9   (952, 1773)
e   0.566666666667   (975, 1773)
t   0.7   (988, 1775)
B   0.8   (1014, 1777)
a   0.566666666667   (1034, 1773)
n   0.566666666667   (1051, 1773)
k   0.833333333333   (1069, 1777)
i   0.833333333333   (1083, 1777)
n   0.566666666667   (1097, 1773)
g   0.833333333333   (1115, 1785)
A   0.8   (227, 1829)
U   0.8   (252, 1829)
T   0.8   (277, 1829)
O   0.833333333333   (300, 1830)
S   0.833333333333   (323, 1829)
W   1.06666666667   (350, 1829)
E   0.8   (377, 1829)
E   0.8   (399, 1829)
P   0.8   (419, 1829)
-   0.333333333333   (461, 1813)
T   0.833333333333   (495, 1830)
r   0.566666666667   (513, 1825)
a   0.6   (526, 1827)
n   0.6   (543, 1827)
s   0.6   (559, 1827)
f   0.866666666667   (574, 1831)
e   0.6   (586, 1827)
r   0.6   (600, 1827)
t   0.7   (619, 1827)
o   0.6   (633, 1827)
l   0.866666666667   (656, 1831)
i   0.866666666667   (666, 1831)
n   0.566666666667   (680, 1826)
k   0.866666666667   (698,

c   0.6   (506, 2471)
o   0.6   (523, 2471)
l   0.866666666667   (536, 2475)
l   0.833333333333   (546, 2474)
e   0.566666666667   (559, 2470)
c   0.566666666667   (575, 2470)
t   0.7   (588, 2472)
e   0.566666666667   (601, 2470)
d   0.9   (618, 2476)
n   0.866666666667   (648, 2475)
o   0.6   (669, 2472)
m   0.9   (691, 2470)
t   0.7   (719, 2472)
h   0.833333333333   (733, 2474)
e   0.6   (751, 2472)
c   0.6   (775, 2472)
u   0.6   (792, 2472)
s   0.6   (808, 2472)
t   0.7   (820, 2472)
o   0.6   (834, 2472)
m   0.833333333333   (857, 2472)
e   0.6   (879, 2472)
r   0.6   (894, 2472)
W   3.0   (1053, 2589)
E   0.933333333333   (1121, 2596)
n   0.666666666667   (1145, 2592)
d   0.933333333333   (1167, 2596)
o   0.7   (1198, 2592)
f   0.933333333333   (1216, 2596)
S   0.966666666667   (1242, 2596)
t   0.866666666667   (1260, 2595)
a   0.7   (1276, 2592)
t   0.866666666667   (1293, 2595)
e   0.666666666667   (1308, 2592)
m   1.03333333333   (1333, 2592)
e   0.666666666667   (1359, 2592

True

In [78]:
#pytesseract.pytesseract??